# Grand Theft Auto V Driving learning with Deep Learning (CNN+RNN)
---
Self driving car in Grand Theft Auto V with Deep Learning.
https://github.com/eritzyg/GTAV-Self-driving-car

### Authors: Iker García and Eritz Yerga

See [this README](https://github.com/eritzyg/GTAV-Self-driving-car/blob/master/Notebook/README.md) for info about the notebook and required libraries.

---
## Index:

1. <a href="#1.-Generate-dataset">Generate dataset</a>
    * <a href="#Frame-capture-functions">Frame capture functions</a>
    * <a href="#Image-preprocessing-functions">Image preprocess functions</a>
    * <a href="#Game-control-and-input-reading-functions">Game control and input reading functions</a>
    * <b><a href="#Generate-dataset">Generate dataset</a></b>
2. <a href="#2.-Dataset-processing-utilities">Dataset processing utilities</a>
3. <a href="#3.-Define-the-model">Define our model</a>
4. <a href="#4.-Train">Train</a>
5. <a href="#5.-Run-our-model-in-the-game">Run our model in the game</a>

<a href=""></a>

---
First we import the libraries we are going to need to run this notebook.

In [ ]:
import numpy as np
from PIL import ImageGrab
import cv2
import time
from sys import stdout
from IPython.display import clear_output
import os
from grabber import Grabber
import threading
import matplotlib.pyplot as plt
from collections import Counter
from random import shuffle
import glob
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import TimeDistributed, LSTM, Flatten, Dense, InputLayer, MaxPooling2D, Dropout, Activation, Embedding, GRU, ConvLSTM2D
from keras.layers.convolutional import Convolution2D
from keras import optimizers
from keras.models import load_model
from keras import initializers
import h5py
import log
from heapq import nlargest

# 1. Generate dataset

We will generate the dataset for training the model later on, it is important that we set the game on the first person view and take into account certain conditions for the dataset. Check the "Generation of the dataset" section in the [documentation](https://github.com/eritzyg/GTAV-Self-driving-car#documentation) for more information on this.

### Frame capture functions
These functions capture the game's frames in 1600x900 Windowed mode.

Screen record is the method to get one frame and img_thread is the thread we will later use to constantly capture the game's output.

In [ ]:
global grb
grb = Grabber(bbox=(1,26,1601,926))
def screen_record(method = 'grabber'):
    if method == 'ImameGrab':
        printscreen =  ImageGrab.grab(bbox=(1,26,1601,926))
        generalIMG = np.array(printscreen)
    
    elif method == 'grabber':
        global grb
        printscreen = None
        printscreen = grb.grab(printscreen)
        generalIMG = np.array(printscreen)
    
    return generalIMG          

In [ ]:
global front_buffer
global back_buffer
front_buffer = np.zeros((1600, 900), dtype=np.int8)
back_buffer = np.zeros((1600, 900), dtype=np.int8)

global fps
fps = 0

def img_thread():
    global front_buffer
    global back_buffer
    global fps
    
    last_time = time.time()
    while True:
        front_buffer = screen_record()
        # Swap buffers
        front_buffer, back_buffer = back_buffer, front_buffer
        fps = int(1.0/(time.time()-last_time))
        last_time = time.time()
    return
    

### Image preprocessing functions

We will define a function that will apply the preprocessing we want to the images:

In [ ]:
def preprocess_image(image):
    proccessed_image = cv2.resize(image,(480,270))
    
    return proccessed_image

### Game control and input reading functions

These functions will read the inputs and generate a array for later use when generating the dataset:

In [ ]:
from game_control import PressKey, ReleaseKey
from getkeys import key_check

In [ ]:
def keys_to_output(keys):
    '''
    Convert keys to a ...multi-hot... array

    [A,W,D] boolean values.
    '''
    output = [0,0,0,0]
    
    if 'A' in keys:
        output[0] = 1
    if 'D' in keys:
        output[1] = 1
    if 'W' in keys:
        output[2] = 1
    if 'S' in keys:
        output[3] = 1
    
    return output

This sequencer thread will capture the sequences of 5 frames with a separation of 1/capturerate ms each.

In [ ]:
global seq
global num
num = 0
seq = []

global key_out
key_out = [0, 0, 0, 0]

def image_sequencer_thread():
    global back_buffer
    global seq
    global key_out
    global num
    
    # Frames per second capture rate
    capturerate = 10.0
    while True:
        last_time = time.time()
        if len(seq) == 5:
            del seq[0]

        seq.append(preprocess_image(np.copy(back_buffer)))
        num = num + 1
        keys = key_check()
        key_out = keys_to_output(keys)
        waittime = (1.0/capturerate)-(time.time()-last_time)
        if waittime>0.0:
            time.sleep(waittime)

This function will be useful to check which class corresponds the input we captured to:

In [ ]:
def counter_keys(key):
        if np.array_equal(key , [0,0,0,0]):
            return 0
        elif np.array_equal(key , [1,0,0,0]):
            return 1
        elif np.array_equal(key , [0,1,0,0]):
            return 2
        elif np.array_equal(key , [0,0,1,0]):
            return 3
        elif np.array_equal(key , [0,0,0,1]):
            return 4
        elif np.array_equal(key , [1,0,1,0]):
            return 5
        elif np.array_equal(key , [1,0,0,1]):
            return 6
        elif np.array_equal(key , [0,1,1,0]):
            return 7
        elif np.array_equal(key , [0,1,0,1]):
            return 8
        else:
            return -1

This is the function that the data saving threads will run to save the dataset to compressed files (change the path if needed):

In [ ]:
def save_data(data,number):
    file_name = 'E:\\GTAV_AI\\training_data'+str(number)+'.npz'
    np.savez_compressed(file_name,data)
    del data

## Generate dataset

We will run this function to generate the dataset:

In [ ]:
def run():
    global fps
    global front_buffer
    global back_buffer
    global seq
    global key_out
    global num
    training_data = []
    threads = list()
    th_img = threading.Thread(target=img_thread)
    th_seq = threading.Thread(target=image_sequencer_thread)
    threads.append(th_img)
    threads.append(th_seq)
    th_img.start()
    time.sleep(1)
    th_seq.start()
    l = 0
    fn = 0
    time.sleep(4)
    last_num = 0
    
    number_of_keys = [0,0,0,0,0,0,0,0,0]
    
    while True:
        img_seq = seq.copy()
        output = key_out.copy()
        
        while len(img_seq) != 5 or last_num==num:
            del img_seq, output
            img_seq = seq.copy()
            output = key_out.copy()
        last_num = num
        
        clear_output(wait=True)
        stdout.write('Recording at {} FPS \n'.format(fps))
        stdout.write('Images in sequence {} \n'.format(len(img_seq)))
        stdout.write('Training data len {} secuences \n'.format(l))
        stdout.write('Number of archives {}\n'.format(fn))
        stdout.write('Keys pressed: ' + str(output) + ' \n')
        stdout.write('Keys samples in this file: ' + 'none:' + str(number_of_keys[0]) + ' A:' + str(number_of_keys[1])+ ' D:' + str(number_of_keys[2]) + ' W:' + str(number_of_keys[3])+ ' S:' + str(number_of_keys[4]) + ' AW:'  + str(number_of_keys[5]) + ' AS:' + str(number_of_keys[6]) + ' WD:' + str(number_of_keys[7]) + ' SD:' + str(number_of_keys[8]) + ' \n')
        stdout.flush()
        
        key  = counter_keys(output)
        
        if key != -1:
            larg = nlargest(9,number_of_keys)
            prop = (9. - float(larg.index(number_of_keys[key])))/10
            if(number_of_keys[key]  > np.mean(number_of_keys) * 1.25):
                prop = prop + 0.05
            if (np.random.rand() > prop):
                number_of_keys[key] += 1
                l = l+1
                training_data.append([img_seq[0],img_seq[1],img_seq[2],img_seq[3],img_seq[4], output])
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

        if len(training_data) % 500 == 0:
            print(len(training_data))
            threading.Thread(target=save_data, args=(training_data.copy(), fn,)).start()
            fn = fn + 1
            del training_data
            training_data = []
            


Now we will run the capture and generation of the dataset, remember that the game's window must be in 1600x900 resolution and located in the top left part of the screen ((0,0) coordinates).

Once you captured all the data you need you can interrupt the kernel to stop the run function.

In [ ]:
time.sleep(5)
run()

# 2. Dataset processing utilities
Some of these functions are not used, but we include them for debug purposes. Others are used later on.

We can plot some data to see if it captured correctly:

In [ ]:
plotImgs = False
if plotImgs:
    with np.load('E:\\GTAV_AI\\filename.npz') as data:
        training_data = data['arr_0']
    i= 502
    plt.imshow(cv2.cvtColor(training_data[i][0], cv2.COLOR_BGR2RGB))
    plt.show()
    plt.imshow(cv2.cvtColor(training_data[i][1], cv2.COLOR_BGR2RGB))
    plt.show()
    plt.imshow(cv2.cvtColor(training_data[i][2], cv2.COLOR_BGR2RGB))
    plt.show()
    plt.imshow(cv2.cvtColor(training_data[i][3], cv2.COLOR_BGR2RGB))
    plt.show()
    plt.imshow(cv2.cvtColor(training_data[i][4], cv2.COLOR_BGR2RGB))
    plt.show()

This function is used to count the number of instances per class in a set:

In [ ]:
def number_instances_per_class(data):
    
    nonekey = []
    A = []
    D = []
    W = []
    S = []
    
    AD = []
    AW = []
    AS = []
    DW =[]
    DS = []
    WS =[]
    
    ADW = []
    AWS =[]
    ADS = []
    DWS = []
    
    ASWS = []
    
    np.random.shuffle(data)
    
    for d in data:
        if np.array_equal(d[5] , [0,0,0,0]):
            nonekey.append(d)
        elif np.array_equal(d[5] , [1,0,0,0]):
            A.append(d)
        elif np.array_equal(d[5] , [0,1,0,0]):
            D.append(d)
        elif np.array_equal(d[5] , [0,0,1,0]):
            W.append(d)
        elif np.array_equal(d[5] , [0,0,0,1]):
            S.append(d)
        elif np.array_equal(d[5] , [1,1,0,0]):
            AD.append(d)
        elif np.array_equal(d[5] , [1,0,1,0]):
            AW.append(d)
        elif np.array_equal(d[5] , [1,0,0,1]):
            AS.append(d)
        elif np.array_equal(d[5] , [0,1,1,0]):
            DW.append(d)
        elif np.array_equal(d[5] , [0,1,0,1]):
            DS.append(d)
        elif np.array_equal(d[5] , [0,0,1,1]):
            WS.append(d)
        elif np.array_equal(d[5] , [1,1,1,0]):
            ADW.append(d)
        elif np.array_equal(d[5] , [1,1,0,1]):
            AWS.append(d)
        elif np.array_equal(d[5] , [1,1,0,1]):
            ADS.append(d)
        elif np.array_equal(d[5] , [0,1,1,1]):
            DWS.append(d)
        elif np.array_equal(d[5] , [1,1,1,1]):
            ASWS.append(d)
    return [nonekey,A,D,W,S,AW,AS,DW,DS]
    


This function will balance the number of instances of the classes in a set by deleting extra instances after shuffling:

In [ ]:
def balance_data(data_in_clases):
    balanced_data = []
    data_in_clases.sort(key=len)
    max_len = len(data_in_clases[0])
        
    for data in data_in_clases:
        if len(data) > max_len:
            data=data[:max_len]
        for d in data:
            balanced_data.append(d)
        
    np.random.shuffle(balanced_data)
    
    return balanced_data
    

For debug purposes:

In [ ]:
debug = False
if debug:
    with np.load('E:\\GTAV_AI\\training_data'+str(20)+'.npz') as data:
         training_data = data['arr_0']
    number = number_instances_per_class(training_data)
    print('none: ' + str(len(number[0])))
    print('A: ' + str(len(number[1])))
    print('D ' + str(len(number[2])))
    print('W ' + str(len(number[3])))
    print('S ' + str(len(number[4])))
    print('AW ' + str(len(number[5])))
    print('AS ' + str(len(number[6])))
    print('DW ' + str(len(number[7])))
    print('DS ' + str(len(number[8])))

    balanced_data = balance_data(number)
    number = number_instances_per_class(balanced_data)
    print('none: ' + str(len(number[0])))
    print('A: ' + str(len(number[1])))
    print('D ' + str(len(number[2])))
    print('W ' + str(len(number[3])))
    print('S ' + str(len(number[4])))
    print('AW ' + str(len(number[5])))
    print('AS ' + str(len(number[6])))
    print('DW ' + str(len(number[7])))
    print('DS ' + str(len(number[8])))


# 3. Define the model

We will define the model to use in our training, please select the model you want to use in the cell bellow (the models are described in the "Definition of the DNN" of the [documentation](https://github.com/eritzyg/GTAV-Self-driving-car#documentation)):

In [ ]:
# Select the model to use:
# 'CNN+MPL' : Convolutional neural network with multi layer perceptron.
# 'CNN+RNN' : Convolutional neural network with recurrent neural network.
selected_model = 'CNN+RNN'

And now we define the corresponding model:

In [ ]:
if selected_model == 'CNN+RNN':
    model = Sequential()

    model.add(InputLayer(input_shape=(5, 270, 480, 3)))

    model.add(TimeDistributed(Convolution2D(32, (4,4), data_format='channels_last')))
    model.add(TimeDistributed(Activation('relu')))
    print(model.output_shape)

    model.add(TimeDistributed(Convolution2D(32, (4,4), data_format='channels_last')))
    model.add(TimeDistributed(Activation('relu')))
    print(model.output_shape)

    model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
    model.add(TimeDistributed(Dropout(0.25)))
    print(model.output_shape)

    model.add(TimeDistributed(Convolution2D(16, (3,3), data_format='channels_last')))
    model.add(TimeDistributed(Activation('relu')))
    print(model.output_shape)


    model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
    model.add(TimeDistributed(Dropout(0.25)))
    print(model.output_shape)

    model.add(TimeDistributed(Flatten()))
    print(model.output_shape)

    model.add(GRU(256, kernel_initializer=initializers.RandomNormal(stddev=0.001))) #128
    model.add(Dropout(0.25))
    print(model.output_shape)

    model.add(Dense(100))
    print(model.output_shape)

    model.add(Dense(80))
    print(model.output_shape)

    model.add(Dense(40))
    print(model.output_shape)

    model.add(Dense(9, activation='sigmoid'))
    print(model.output_shape)

    opt = optimizers.rmsprop(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy']) 

In [ ]:
if selected_model == 'CNN+MLP':
    model = Sequential()

    model.add(InputLayer(input_shape=(5, 270, 480, 3)))

    model.add(TimeDistributed(Convolution2D(16, (4,8), data_format='channels_last')))
    model.add(TimeDistributed(Activation('relu')))
    print(model.output_shape)

    model.add(TimeDistributed(Convolution2D(16, (4,4), data_format='channels_last')))
    model.add(TimeDistributed(Activation('relu')))
    print(model.output_shape)

    model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
    model.add(TimeDistributed(Dropout(0.25)))
    print(model.output_shape)

    model.add(TimeDistributed(Convolution2D(12, (3,3), data_format='channels_last')))
    model.add(TimeDistributed(Activation('relu')))
    print(model.output_shape)

    model.add(TimeDistributed(MaxPooling2D(pool_size=(5, 5), data_format='channels_last')))
    model.add(TimeDistributed(Dropout(0.25)))
    print(model.output_shape)

    model.add(Flatten())
    print(model.output_shape)

    model.add(Dense(300))
    print(model.output_shape)
    model.add(Dense(100))
    print(model.output_shape)
    print(model.output_shape)
    model.add(Dense(9, activation='sigmoid'))
    print(model.output_shape)

    opt = optimizers.rmsprop(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy']) 

# 4. Train

Now we will define some functions to reshape the data according to the input of our model:

In [ ]:
def reshape_custom_X(data, verbose = 1):
    reshaped = np.zeros((data.shape[0], 5, 270, 480, 3), dtype=np.float32)
    for i in range(0, data.shape[0]):
        for j in range(0, 5):
            if (verbose == 1):
                clear_output(wait=True)
                stdout.write('Reshaped image: ' + str(i))
                stdout.flush()
            reshaped[i][j] = data[i][j]/255.
            
    return reshaped

def reshape_custom_y(data):
    reshaped = np.zeros((data.shape[0], 9), dtype=np.float32)
    for i in range(0, data.shape[0]):
            if np.array_equal(data[i][0] , [0,0,0,0]):
                reshaped[i][0] = 1.
            elif np.array_equal(data[i][0] , [1,0,0,0]):
                reshaped[i][1] = 1.
            elif np.array_equal(data[i][0] , [0,1,0,0]):
                reshaped[i][2] = 1.
            elif np.array_equal(data[i][0] , [0,0,1,0]):
                reshaped[i][3] = 1.
            elif np.array_equal(data[i][0] , [0,0,0,1]):
                reshaped[i][4] = 1.
            elif np.array_equal(data[i][0] , [1,0,1,0]):
                reshaped[i][5] = 1.
            elif np.array_equal(data[i][0] , [1,0,0,1]):
                reshaped[i][6] = 1.
            elif np.array_equal(data[i][0] , [0,1,1,0]):
                reshaped[i][7] = 1.
            elif np.array_equal(data[i][0] , [0,1,0,1]):
                reshaped[i][8] = 1.
    return reshaped

These functions will help for picking the number of batches and the start and end indexes:

In [ ]:
def get_num_batches(length, BATCH_SIZE):
    if (int(length/BATCH_SIZE)*BATCH_SIZE == length):
        return int(length/BATCH_SIZE)
    else:
        return int(length/BATCH_SIZE)+1

def get_start_end(iteration, BATCH_SIZE, max_length):
    start = iteration*BATCH_SIZE
    if (start > max_length):
        print("ERROR: Check iterations made! Must be wrong")
        return -1, -1
    end = (iteration+1)*BATCH_SIZE
    if (end > max_length):
        end = max_length
    return start, end

And now we run the training process, the accuracy can be seen after each epoch (all the inputs will be saved to a "log.txt" file, so don't worry if you miss out some output). Also remember to change the path to the same path where you saved the dataset:

In [ ]:
log.openlog()

# Define the sizes and epoches
BATCH_SIZE = 10
TEST_BATCH_SIZE = 10
n_epochs = 6

# This path will be the path used to load the dataset files!
files = glob.glob("E:\\GTAV_AI\\merged\\*.npz")


actual_file = 0
acc_for_files = []
for fil in files:
    actual_file = actual_file + 1
    log.output("\n Loading input: "+fil)
    with np.load(fil) as data:
         training_data = data['arr_0']

    #print("\n Balancing data...")
    #number = number_instances_per_class(training_data)
    #training_data = np.array(balance_data(number))

    log.output("\n Reshaping data...")
    np.random.shuffle(training_data)
    train = training_data[0:int(len(training_data)*0.90)]
    test = training_data[int(len(training_data)*0.90):len(training_data)]
    del training_data
    X_train = reshape_custom_X(train[:, 0:5])
    y_train = reshape_custom_y(train[:, 5:6])
    X_test = reshape_custom_X(test[:, 0:5])
    y_test= reshape_custom_y(test[:, 5:6])
    del train, test
    log.output("\n Training...")
    data_length = len(X_train)
    n_batch = get_num_batches(data_length, BATCH_SIZE)
    for epoch in range(n_epochs):
        for iteration in range(n_batch):
            start, end = get_start_end(iteration, BATCH_SIZE, data_length)
            model.fit(x=X_train[start:end], y=y_train[start:end], epochs=1, verbose=0)
            clear_output(wait=True)
            log.output('\n => File : ' + str(actual_file) + ' of ' + str(len(files)))
            log.output('\n ==> EPOCH : ' + str(epoch+1) + ' of ' + str(n_epochs))
            log.output('\n ===> Iteration: ' + str(iteration+1) + ' of ' + str(n_batch))
            #score = model.evaluate(X_train[start:end], y_train[start:end], verbose=0)
            #log.output("\n Train batch accuracy: %.2f%%" % (score[1]*100))
            stdout.flush()
        prec = np.zeros((9))
        for i in range(len(y_test)):
            p = model.predict_classes(X_test[i:i+1])
            prec[p] = prec[p] + 1
        log.output("\n ==> Predictions:"+str(prec))
        stdout.flush()   
    total_acc = 0.0
    num = 0
    data_length = len(X_test)
    n_batch = get_num_batches(data_length, TEST_BATCH_SIZE)
    for iteration in range(n_batch):
        start, end = get_start_end(iteration, TEST_BATCH_SIZE, data_length)
        score = model.evaluate(X_test[start:end], y_test[start:end], verbose=0)
        log.output("\n => Batch %s: %.2f%%" % (model.metrics_names[1], score[1]*100))
        total_acc = total_acc + score[1]
        num = num + 1
    total_acc = total_acc/float(num)
    log.output("\n ==> Total acc for file %s: %.2f%%" % (fil, total_acc*100))
    acc_for_files.append(total_acc*100)


    prec = np.zeros((9))
    for i in range(len(y_test)):
        p = model.predict_classes(X_test[i:i+1])
        prec[p] = prec[p] + 1
    log.output("\n ==> Predictions:"+str(prec))
    time.sleep(5)

for acc in range(len(acc_for_files)):
    log.output("\n ==> Total acc after file %d: %.2f%%" % (acc, acc_for_files[acc]))
    
    
log.closelog()

Now we save the learned network:

In [ ]:
import h5py as h5py
model.save('E:\\GTAV_AI\\'+selected_model)

# 5. Run our model in the game

Now it is time to test our model in the game, we will reduce the amount of VRAM tensorflow can use so that the game has some VRAM in spare. We will also import the library needed to send inputs to the game:

In [ ]:
# Let the game have some VRAM (needed or the game will crash)
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
set_session(tf.Session(config=config))

# For controlling the game
from inputsHandler import select_key
from tkinter import *

In [ ]:
def key_press(key):
    if key == 1:
        return'A'
    if key == 2:
        return'D'
    if key == 3:
        return'W'
    if key == 4:
        return'S'
    if key == 5:
        return'AW'
    if key == 6:
        return'AS'
    if key == 7:
        return'DW'
    if key == 8:
        return'DS'
    return 'none'

This fuction calculates the Mean Squared Error between 2 images. Is used to detect if the car is stuck somewhere. 

In [ ]:
def mse(imageA, imageB):

    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

We define the run function for our network:

In [ ]:
#Run Configurations
show_current_control = False #It will show a windows with a message indicating if the car is currently be controlled by
                            #Network  or by a Human
    
show_whatAIsees = False #It will show the 5 images that the netowrk uses the predict the output 

enable_evasion = False #If the program detects that the car is not moving (for example because it is stuck facing a wall and
                        #the network is not able to return to the road) It will make the car move backwards for a second.

In [ ]:
def run_IA():
    global fps
    global front_buffer
    global back_buffer
    global seq
    global key_out
    global num
    
    model = load_model('E:\\GTAV_AI\\'+selected_model)
    
    training_data = []
    threads = list()
    th_img = threading.Thread(target=img_thread)
    th_seq = threading.Thread(target=image_sequencer_thread)
    threads.append(th_img)
    threads.append(th_seq)
    th_img.start()
    time.sleep(1)
    th_seq.start()
    time.sleep(4)
    
    last_num = 0
    
    last_time = time.time()
    
    if show_current_control:
        root = Tk()
        var = StringVar()
        var.set('IA CONDUCIENDO')
        l = Label(root, textvariable = var, fg='green', font=("Courier", 44))
        l.pack()

    
    while True:
       
        img_seq = seq.copy()
        while len(img_seq) != 5 or last_num==num:
            del img_seq
            img_seq = seq.copy()
        last_num = num
        array=[]
        array.append([img_seq[0],img_seq[1],img_seq[2],img_seq[3],img_seq[4]])
        NNinput = np.array(array)
        
        x = reshape_custom_X(NNinput[:,0:5],0)
        p = model.predict_classes(x)
        
        if not 'J' in key_check():
            select_key(p[0])
            if show_current_control:
                var.set('IA CONDUCIENDO')
                l.config(fg='green')
                root.update()
        else:
            if show_current_control:
                var.set('CONTROL MANUAL')
                l.config(fg='red')
                root.update()

        #This is used to detect if the car is stuck somewhere (for example facing a wall) and the network does not know what to do. It will move the car
        #backward for a second.
        
        if enable_evasion:
            score = mse(img_seq[0],img_seq[4])
            if score < 1000:
                if show_current_control:
                    var.set('MANIOBRA DE EVASIÓN')
                    l.config(fg='blue')
                    root.update()
                select_key(4)
                time.sleep(1)
                if np.random.rand()>0.5:
                    select_key(6)
                else:
                    select_key(8)
                time.sleep(0.2)
                if show_current_control:
                    var.set('IA CONDUCIENDO')
                    l.config(fg='green')
                    root.update()

        time_act = time.time()
        clear_output(wait=True)
        stdout.write('Recording at {} FPS \n'.format(fps))
        stdout.write('Images in sequence {} \n'.format(len(img_seq)))
        stdout.write('Keys pressed: ' + key_press(p[0]) + '\n')
        stdout.write('Actions per second: ' + str(1/(time_act-last_time)) + '\n')
        if enable_evasion:
            stdout.write('Diference from img 1 to img 5: ' + str(score))
        stdout.flush()
        last_time = time.time()
        
        if show_whatAIsees:
            cv2.imshow('window1',np.array(img_seq[0])) 
            cv2.imshow('window2',np.array(img_seq[1]))
            cv2.imshow('window3',np.array(img_seq[2]))
            cv2.imshow('window4',np.array(img_seq[3]))
            cv2.imshow('window5',np.array(img_seq[4]))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
     

And now we run the network in the game:

In [ ]:
run_IA()